In [ ]:
#Cell_1.
import openai
import re
import os
from openai import OpenAI

oai = OpenAI(
    api_key = 'XXXX',
    base_url = "https://api.openai.iniad.org/api/v1"
)

In [ ]:
#Cell_2.
with open('world_country_population.csv', 'r', encoding='utf-8') as file:
       file_content = file.read()

In [ ]:
#Cell_3.
messages=[
       {"role": "system", "content": "あなたはデータサイエンティストです。"},
       {"role": "user", "content": "いまアップロードしたデータ'world_country_population.csv'について解説してください。"},
       {"role": "user", "content": file_content}
]

resp = oai.chat.completions.create(
    model='gpt-4o-mini',
    messages=messages,
)

print(resp.choices[0].message.content)

In [ ]:
#Cell_4.
messages.append({"role": "assistant", "content": resp.choices[0].message.content})
messages.append({"role": "user", "content": "有難うございます。そうしましたら、このworld_country_population.csvから2019年時点の中国、インド、インドネシアの人口を取得して、世界地図上に色付けしてください。"})

resp = oai.chat.completions.create(
    model='gpt-4o-mini',
    messages=messages,
)

print(resp.choices[0].message.content)

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

# csvファイルの読み込み
data = pd.read_csv("world_country_population.csv",sep=",",skiprows=4,header=0)


# 2019年のデータを取得（中国、インド、インドネシア）
population_2019 = data.loc[data['Country Name'].isin(['China', 'India', 'Indonesia']), ['Country Name', '2019']]

# GeoPandasで世界地図データを読み込む
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# 人口データを地図データにマージ
world = world.merge(population_2019, how='left', left_on='name', right_on='Country Name')

# 色の設定
world['color'] = 'lightgrey'  # デフォルトの色
world.loc[world['Country Name'] == 'China', 'color'] = 'red'  # 中国
world.loc[world['Country Name'] == 'India', 'color'] = 'blue'  # インド
world.loc[world['Country Name'] == 'Indonesia', 'color'] = 'green'  # インドネシア

# 地図をプロット
fig, ax = plt.subplots(1, figsize=(15, 10))
world.boundary.plot(ax=ax, linewidth=1)
world.plot(column='color', ax=ax, legend=True, edgecolor='black')

# タイトル設定
plt.title('Population of China, India, and Indonesia in 2019', fontsize=15)
plt.show()